## Exercise 2
Track the evolution of the electron energy distribution for different strengths of the synchrotron and inverse Compton cooling. Plot the distributions as a function of the electron kinetic energy ($\gamma - 1$) in log-log scale with logarithmic bins for the energy.

In [ ]:
from wrapper_sim import Simulation
import pusher
import numpy as np
from matplotlib import figure, axes, patches, lines, colors
from typing import Optional
from warnings import warn
import matplotlib.pyplot as plt

In [ ]:
particles = int(1e5)
gd = np.geomspace(1,1e3,4)
cs = [{gsyn:gic for gic in gd} for gsyn in gd]
saves = 10
temp = 1
names = []

for gamma_drag in cs:
    sim = Simulation()
    sim.begin(particles, temp, gamma_drag, saves)
    sim.run()
    name = f"P1e5-T1-S10-syn{gamma_drag['syn']}-ic{gamma_drag['ic']}"
    sim.end(name)
    names.append(name)

In [ ]:
def axhist_energy_spectrum(ax: axes.Axes, U: np.ndarray, color: str, *args, **kwargs):
    gm1 = pusher.lorentz_factor(U) - 1
    opacity = np.linspace(0, 1, gm1.shape[0])

    for Ek, alpha in zip(gm1, opacity):
        l = ax.hist(np.log10(Ek), histtype="step", bins="fd",
                    alpha=alpha, color=color, *args, **kwargs)

    return ax, l


def exercise2(names: list[str]):
    sim = Simulation()

    fig = plt.figure()
    ax = fig.add_subplot()

    handles = []
    labels = []
    for name, color in zip(names, colors.TABLEAU_COLORS.values()):
        sim.load(name)

        ax, l = axhist_energy_spectrum(ax, sim.vel_history, color)
    
        handles.append(l)
        labels.append(fr"$\gamma_\mathrm{{syn}} = {sim.gamma_drag['syn']}, \gamma_\mathrm{{IC}} = {sim.gamma_drag['ic']}$")
    
    ax.set_xlabel(r"$\log_{10}\left(\gamma - 1\right)$")
    ax.set_ylabel("Counts")
    ax.set_yscale("log")

    fig.legend(handles, labels)

    return fig, ax

In [ ]:
figs = []
for i in range(4):
    figs.append(exercise2(names[4*i, 4*(i+1)]))
for f in figs:
    f.show()